# Import raw data
wget -O tree_census_2015.csv https://data.cityofnewyork.us/api/views/uvpi-gqnh/rows.csv?accessType=DOWNLOAD

In [25]:
import pandas as pd
import geopandas

In [26]:
df = pd.read_csv("/home/myn/LHL googledrive/tree_census_2015.csv") # update me with the path to your raw data file (no formatting requried)

# https://cityofnewyork.github.io/opendatatsm/citystandards.html states all geospatial data is in EPSG:3857.
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df["longitude"], df["latitude"], crs="EPSG:3857")
)

# Get some info about the dataset

In [27]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 46 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   tree_id           683788 non-null  int64   
 1   block_id          683788 non-null  int64   
 2   created_at        683788 non-null  object  
 3   tree_dbh          683788 non-null  int64   
 4   stump_diam        683788 non-null  int64   
 5   curb_loc          683788 non-null  object  
 6   status            683788 non-null  object  
 7   health            652172 non-null  object  
 8   spc_latin         652169 non-null  object  
 9   spc_common        652169 non-null  object  
 10  steward           652173 non-null  object  
 11  guards            652172 non-null  object  
 12  sidewalk          652172 non-null  object  
 13  user_type         683788 non-null  object  
 14  problems          652124 non-null  object  
 15  root_stone        683788 non-null  object  

In [28]:
gdf.isna().any()

tree_id             False
block_id            False
created_at          False
tree_dbh            False
stump_diam          False
curb_loc            False
status              False
health               True
spc_latin            True
spc_common           True
steward              True
guards               True
sidewalk             True
user_type           False
problems             True
root_stone          False
root_grate          False
root_other          False
trunk_wire          False
trnk_light          False
trnk_other          False
brch_light          False
brch_shoe           False
brch_other          False
address             False
postcode            False
zip_city            False
community board     False
borocode            False
borough             False
cncldist            False
st_assem            False
st_senate           False
nta                 False
nta_name            False
boro_ct             False
state               False
latitude            False
longitude   

In [29]:
df["spc_common"].value_counts(dropna=False).head(20).index.values

array(['London planetree', 'honeylocust', 'Callery pear', 'pin oak',
       'Norway maple', nan, 'littleleaf linden', 'cherry',
       'Japanese zelkova', 'ginkgo', 'Sophora', 'red maple', 'green ash',
       'American linden', 'silver maple', 'sweetgum', 'northern red oak',
       'silver linden', 'American elm', 'maple'], dtype=object)

In [30]:
gdf[["longitude", "latitude", "geometry"]].loc[gdf["spc_common"] == "Norway maple"]

,longitude,latitude,geometry
331,-73.893711,40.847658,POINT (-73.894 40.848)
333,-73.928171,40.832306,POINT (-73.928 40.832)
335,-73.904082,40.708712,POINT (-73.904 40.709)
336,-73.881538,40.875677,POINT (-73.882 40.876)
337,-73.978833,40.674917,POINT (-73.979 40.675)
...,...,...,...
680285,-73.979225,40.675069,POINT (-73.979 40.675)
680286,-73.995278,40.687196,POINT (-73.995 40.687)
680287,-73.887025,40.850751,POINT (-73.887 40.851)
680288,-73.740067,40.601315,POINT (-73.740 40.601)


# Calculate area of NYC
Ended up not being required but was fun :)

In [31]:
nyc = geopandas.GeoDataFrame.from_file("/home/myn/LHL googledrive/old_PUMA_or_Subborough.geo.json")
nyc = nyc.to_crs({'proj': 'cea'})
nyc_area = nyc.area.sum()/10**6
nyc_area  # https://www.google.com/search?q=nyc+area agrees to publized area approx.

780.9194543793957

# Density based clustering
*spc_common* holds the common species name for each tree. Approx 5% of the rows for this field is blank which isn't material for our purpose of finding tree density clusters per species.

In [32]:
# https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS

import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np


def cluster_DBSCAN(
    radius_km: float,
    min_samples_pct: float,
    plot: bool,
    species: str | None = None,
) -> pd.DataFrame:
    trees_df = gdf.loc[:, ["longitude", "latitude", "geometry"]]

    if species:
        trees_df = trees_df.loc[gdf["spc_common"] == species]

    # All distance calculations are done in radians because a degree represents a different physical distance depending on where on earth you are.
    coords = np.radians(trees_df[["longitude", "latitude"]].values)
    kms_per_radian = 6371.0088  # physical constant
    epsilon = radius_km / kms_per_radian
    min_samples = int(len(trees_df) * min_samples_pct)

    cluster = DBSCAN(
        eps=epsilon,
        metric="haversine",
        min_samples=min_samples,
        n_jobs=-1,
    ).fit(coords)

    cluster_labels = cluster.labels_
    print(f"min_samples: {min_samples}")
    print(f"clusters: {len(set(cluster_labels))}")
    print(f"trees: {len(cluster_labels)}")

    trees_df["cluster"] = cluster_labels

    if plot:
        fig, ax = plt.subplots(1, figsize=(20, 15))
        trees_df.plot(
            categorical=True,
            legend=True,
            column="cluster",
            ax=ax,
            markersize=0.1,
            alpha=0.8,
            cmap="turbo",
        )
        ax.axis("off")
        ax.set_title(f"{species} clusters", fontsize=20)
        plt.show()
    
    return trees_df

# function for OPTICS clustering but was not used.
def cluster_OPTICS(species: str, plot: bool, min_samples: float):
    trees_df = gdf[["longitude", "latitude", "geometry"]].loc[
        gdf["spc_common"] == species
    ]
    coords = trees_df[["longitude", "latitude"]].values
    cluster = OPTICS(
        metric="haversine",
        min_cluster_size=min_samples,
        n_jobs=-1,
    ).fit(np.radians(coords))
    cluster_labels = cluster.labels_
    trees_df["cluster"] = cluster_labels

    print(pd.Series(cluster_labels).value_counts())
    print(len(set(cluster_labels)))
    print(len(cluster_labels))

    if plot:
        fig, ax = plt.subplots(1, figsize=(20, 15))
        trees_df.plot(
            categorical=True,
            legend=True,
            column="cluster",
            ax=ax,
            markersize=0.1,
            alpha=0.8,
            cmap="turbo",
        )
        ax.axis("off")
        ax.set_title(f"{species} clusters", fontsize=20)
        plt.show()


In [33]:
treeclusters = pd.DataFrame()

for col in df["spc_common"].value_counts().head(20).index.values:
    clusterlabels = cluster_DBSCAN(
        species=col, radius_km=0.5, min_samples_pct=0.01, plot=False
    )  # For reference a Manhattan city block is approx 80m x 274m.
    treeclusters = pd.concat([treeclusters, clusterlabels])

print(treeclusters)

min_samples: 870
clusters: 9
trees: 87014
min_samples: 642
clusters: 4
trees: 64264
min_samples: 589
clusters: 5
trees: 58931
min_samples: 531
clusters: 4
trees: 53185
min_samples: 341
clusters: 10
trees: 34189
min_samples: 297
clusters: 6
trees: 29742
min_samples: 292
clusters: 5
trees: 29279
min_samples: 292
clusters: 5
trees: 29258
min_samples: 210
clusters: 6
trees: 21024
min_samples: 193
clusters: 10
trees: 19338
min_samples: 172
clusters: 7
trees: 17246
min_samples: 162
clusters: 11
trees: 16251
min_samples: 135
clusters: 8
trees: 13530
min_samples: 122
clusters: 7
trees: 12277
min_samples: 106
clusters: 10
trees: 10657
min_samples: 84
clusters: 13
trees: 8400
min_samples: 79
clusters: 9
trees: 7995
min_samples: 79
clusters: 11
trees: 7975
min_samples: 70
clusters: 13
trees: 7080
min_samples: 68
clusters: 12
trees: 6879
        longitude   latitude                geometry  cluster
9      -73.969744  40.586357  POINT (-73.970 40.586)        0
10     -73.911171  40.782428  POINT (-